In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import unsloth
import pandas as pd
import datasets

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


    PyTorch 2.6.0+cu124 with CUDA 1204 (you have 2.8.0+cu126)
    Python  3.12.9 (you have 3.12.11)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


🦥 Unsloth Zoo will now patch everything to make training faster!


In [ ]:
from unsloth import FastModel
import torch

model, tokenizer = FastModel.from_pretrained(
    # model_name = "unsloth/gemma-3-1b-it-unsloth-bnb-4bit",
    # model_name = "unsloth/gemma-3-270m-it",
    model_name = "unsloth/gemma-3-4b-it-unsloth-bnb-4bit",
    # model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit",
    # model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    dtype = None,
    max_seq_length = 2048,
    load_in_4bit = True,
    full_finetuning = False,

)

==((====))==  Unsloth 2025.8.9: Fast Gemma3 patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Using float16 precision for gemma3 won't work! Using float32.


model.safetensors:   0%|          | 0.00/4.56G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/210 [00:00<?, ?B/s]

processor_config.json:   0%|          | 0.00/70.0 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/670 [00:00<?, ?B/s]

In [ ]:
model = FastModel.get_peft_model(
    model,
    r = 128,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 128,
    lora_dropout = 0,
    bias = "none",

    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

Unsloth: Making `base_model.model.model.vision_tower.vision_model` require gradients


In [ ]:
from unsloth.chat_templates import get_chat_template
tokenizer = get_chat_template(
    tokenizer,
    chat_template = "gemma3",
    # chat_template = "llama-3.2",
)

In [ ]:
!curl -L "https://huggingface.co/api/datasets/GEM/web_nlg/parquet/en/train/0.parquet" -o train.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   120  100   120    0     0    350      0 --:--:-- --:--:-- --:--:--   349
100  1307  100  1307    0     0   2177      0 --:--:-- --:--:-- --:--:--  2177
100 2326k  100 2326k    0     0  3383k      0 --:--:-- --:--:-- --:--:-- 3383k


In [ ]:
!curl -L "https://huggingface.co/api/datasets/GEM/web_nlg/parquet/en/test/0.parquet" -o test.parquet

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   119  100   119    0     0    291      0 --:--:-- --:--:-- --:--:--   291
100   285  100   285    0     0    437      0 --:--:-- --:--:-- --:--:--     0
100  462k  100  462k    0     0   698k      0 --:--:-- --:--:-- --:--:--  698k


In [ ]:
train_df = pd.read_parquet("train.parquet", engine="pyarrow").sample(frac=0.1, random_state=42).reset_index(drop=True)
test_df = pd.read_parquet("test.parquet", engine="pyarrow").sample(frac=0.1, random_state=42).head(300).reset_index(drop=True)


In [ ]:
train_df.head()

,gem_id,gem_parent_id,input,target,references,category,webnlg_id
0,web_nlg_en-train-5038,web_nlg_en-train-5038,[A-Rosa_Luna | shipOrdered | 2004-01-22],The A-Rosa Luna ship was ordered on January 22...,[],MeanOfTransportation,train/MeanOfTransportation/1/Id25
1,web_nlg_en-train-5864,web_nlg_en-train-5864,"[Aaron_S._Daggett | birthPlace | Greene,_Maine]","The birth place of Aaron S. Daggett is Greene,...",[],Politician,train/Politician/1/Id12
2,web_nlg_en-train-30896,web_nlg_en-train-30896,"[Houston_Texans | city | Texas, Akeem_Dent | d...","Akeem Dent, who debuted with the Atlanta Falco...",[],Athlete,train/Athlete/5/Id52
3,web_nlg_en-train-14789,web_nlg_en-train-14789,[Al_Asad_Airbase | operatingOrganisation | Uni...,The Al Asad Airbase is situated in the Al Anba...,[],Airport,train/Airport/3/Id59
4,web_nlg_en-train-8014,web_nlg_en-train-8014,[Antwerp_International_Airport | cityServed | ...,"Bart De Wever is the mayor of Antwerp, which i...",[],Airport,train/Airport/2/Id130


In [ ]:
task = """
You are tasked with extracting knowledge triples from text. Each triple must follow this exact structure: ’Subject | Relation | Object’

Rules for formatting triples:
Subject:
A single entity name.
Replace spaces with underscores.
Can be a person, place, organization, or other entity.
Relation:
Always in camelCase.
Represents the property or relationship between subject and object.
Object:
Dates → Normalize to ISO format "YYYY-MM-DD".
Degree or institution → Format as "Institution, Degree Year".
Places (cities, states, countries, regions, airports) → Replace spaces with underscores, preserve commas and accents, no quotes.
Entities (people, organizations, missions, artists, genres, events, equipment/vehicles) → Replace spaces with underscores, preserve hyphens and special characters, no quotes.
Entities with qualifiers or disambiguation → Keep parentheses and underscores, no quotes.
Titles, roles, or string values → Always in double quotes.
Lists of multiple entities → Combine into a single, comma-separated string wrapped in double quotes.
Numeric values → Include as-is, no quotes.
Output:
Return all triples as a Python list of strings.
Each triple must strictly follow the Subject | Relation | Object format.
Ensure consistency: underscores for multi-word names, quotes where required, ISO dates, numeric values as-is.
"""

In [ ]:
train_df["task"] = task
train_df["output"] = train_df["input"].apply(lambda x: list(x))
test_df["output"] = test_df["input"].apply(lambda x: list(x))

In [ ]:
train_df.to_csv("train.csv", encoding="utf-8-sig", index=False)
test_df.to_csv("test.csv", encoding="utf-8-sig", index=False)

In [ ]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files="train.csv")

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def convert_to_chatml(example):
    return {
        "conversations": [
            {"role": "system", "content": example["task"]},
            {"role": "user", "content": example["target"]},
            {"role": "assistant", "content": example["output"]}
        ]
    }

dataset = dataset.map(
    convert_to_chatml
)

Map:   0%|          | 0/3543 [00:00<?, ? examples/s]

In [ ]:
def formatting_prompts_func(examples):
   convos = examples["conversations"]
   texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False).removeprefix('<bos>') for convo in convos]
  #  texts = [tokenizer.apply_chat_template(convo, tokenize = False, add_generation_prompt = False) for convo in convos]
   return { "text" : texts, }

dataset = dataset.map(formatting_prompts_func, batched = True)

Map:   0%|          | 0/3543 [00:00<?, ? examples/s]

In [ ]:
dataset["train"]

Dataset({
    features: ['gem_id', 'gem_parent_id', 'input', 'target', 'references', 'category', 'webnlg_id', 'task', 'output', 'conversations', 'text'],
    num_rows: 3543
})

In [ ]:
from trl import SFTTrainer, SFTConfig
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset["train"],
    eval_dataset = None, # Can set up evaluation!
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 4, # Use GA to mimic batch size!
        warmup_steps = 5,
        num_train_epochs = 4, # Set this for 1 full training run.
        max_steps = 50,
        learning_rate = 7e-5, # Reduce to 2e-5 for long training runs
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir="outputs",
        report_to = "none", # Use this for WandB etc
    ),
)

Unsloth: Switching to float32 training since model cannot work with float16


Unsloth: Tokenizing ["text"] (num_proc=2):   0%|          | 0/3543 [00:00<?, ? examples/s]

In [ ]:
# for gemma
from unsloth.chat_templates import train_on_responses_only
trainer = train_on_responses_only(
    trainer,
    instruction_part = "<start_of_turn>user\n",
    response_part = "<start_of_turn>model\n",
)

# for llama
# from unsloth.chat_templates import train_on_responses_only
# trainer = train_on_responses_only(
#     trainer,
#     instruction_part = "<|start_header_id|>user<|end_header_id|>\n\n",
#     response_part = "<|start_header_id|>assistant<|end_header_id|>\n\n",
# )

Map (num_proc=2):   0%|          | 0/3543 [00:00<?, ? examples/s]

In [ ]:
tokenizer.decode(trainer.train_dataset[100]["input_ids"])

'<bos><start_of_turn>user\n\nYou are tasked with extracting knowledge triples from text. Each triple must follow this exact structure: ’Subject | Relation | Object’\n\nRules for formatting triples:\nSubject:\nA single entity name.\nReplace spaces with underscores.\nCan be a person, place, organization, or other entity.\nRelation:\nAlways in camelCase.\nRepresents the property or relationship between subject and object.\nObject:\nDates → Normalize to ISO format "YYYY-MM-DD".\nDegree or institution → Format as "Institution, Degree Year".\nPlaces (cities, states, countries, regions, airports) → Replace spaces with underscores, preserve commas and accents, no quotes.\nEntities (people, organizations, missions, artists, genres, events, equipment/vehicles) → Replace spaces with underscores, preserve hyphens and special characters, no quotes.\nEntities with qualifiers or disambiguation → Keep parentheses and underscores, no quotes.\nTitles, roles, or string values → Always in double quotes.\n

In [ ]:
tokenizer.decode([tokenizer.pad_token_id if x == -100 else x for x in trainer.train_dataset[100]["labels"]]).replace(tokenizer.pad_token, " ")

"                                                                                                                                                                                                                                                                                                                            ['Amsterdam_Airport_Schiphol | runwayLength | 2014.0']<end_of_turn>\n"

In [ ]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 3,543 | Num Epochs = 1 | Total steps = 50
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 4 x 1) = 16
 "-____-"     Trainable parameters = 262,307,840 of 4,562,387,312 (5.75% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,3.455200
2,3.820700
3,2.176900
4,1.438700
5,1.223600
6,0.776000
7,0.543400
8,0.698500
9,0.578200
10,0.461100


In [ ]:
trainer.save_model("/content/drive/MyDrive/finetuned-models/gemma4b")

In [ ]:
import ast

In [ ]:
responses = []
for i in range(len(test_df)):
    input = test_df["target"].values[i]
    output = test_df["output"].values[i]
    messages = [
        {'role': 'system','content': task},
        {'role': 'user','content': input}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        tokenize = False,
        add_generation_prompt = True, # Must add for generation
    ).removeprefix('<bos>')


    from transformers import TextStreamer
    response = model.generate(
        **tokenizer(text, return_tensors = "pt").to("cuda"),
        max_new_tokens = 1000,
        temperature = 0.1,
        streamer = TextStreamer(tokenizer, skip_prompt = True),
    )
    responses.append(response)



['Brandon_Carter | knownFor | Doomsday_argument']<end_of_turn>
['Piotr_Hallmann | birthPlace | Gdynia', 'Gdynia | country | Poland', 'Gdynia | timeZone | Central_European_Timezone', 'Piotr_Hallmann | weight | 70.308 (kilograms)']<end_of_turn>
['Darinka_Dentcheva | field | Mathematical_optimization', 'Darinka_Dentcheva | influencedBy | Andrzej_Piotr_Ruszczyński']<end_of_turn>
['Istanbul | timeZone | +2_UTC']<end_of_turn>
['Nie_Haisheng | birthDate | 1964-10-13', 'Nie_Haisheng | nationality | People\'s_Republic_of_China']<end_of_turn>
['Ciudad_Ayala | country | Mexico', 'Ciudad_Ayala | populationDensity | 1604.0', 'Ciudad_Ayala | leader | City_manager', 'Ciudad_Ayala | isPartOf | Mexico', 'Ciudad_Ayala | isPartOf | Mexico', 'Ciudad_Ayala | isPartOf | Mexico', 'Ciudad_Ayala | population | 1777539.0', 'Ciudad_Ayala | governmentType | Council-manager_government', 'Ciudad_Ayala | timeZoneOffset | -6.0']<end_of_turn>
['Olga_Bondareva | almaMater | Leningrad_State_University', 'Olga_Bondareva 

In [ ]:
tokenizer.decode(response[0])

'<bos><start_of_turn>user\n\nYou are tasked with extracting knowledge triples from text. Each triple must follow this exact structure: ’Subject | Relation | Object’\n\nRules for formatting triples:\nSubject:\nA single entity name.\nReplace spaces with underscores.\nCan be a person, place, organization, or other entity.\nRelation:\nAlways in camelCase.\nRepresents the property or relationship between subject and object.\nObject:\nDates → Normalize to ISO format "YYYY-MM-DD".\nDegree or institution → Format as "Institution, Degree Year".\nPlaces (cities, states, countries, regions, airports) → Replace spaces with underscores, preserve commas and accents, no quotes.\nEntities (people, organizations, missions, artists, genres, events, equipment/vehicles) → Replace spaces with underscores, preserve hyphens and special characters, no quotes.\nEntities with qualifiers or disambiguation → Keep parentheses and underscores, no quotes.\nTitles, roles, or string values → Always in double quotes.\n

In [ ]:
text_responses = []
for i in responses:
    # text_responses.append(str(tokenizer.decode(i[0])).split("assistant<|end_header_id|>")[1].replace("<|eot_id|>","").strip())
    text_responses.append(str(tokenizer.decode(i[0])).split("<start_of_turn>model")[1].replace("<end_of_turn>","").strip())

In [ ]:
test_df["generated_response"] = text_responses

In [ ]:
test_df.to_csv("/content/drive/MyDrive/finetuned-models-results/gemma4b-test_results.csv", encoding="utf-8-sig", index=False)